## Data Analysis

In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

train_df = pd.read_csv('/mnt/c/wo_pessoal/uber_assessment/data_original/latam_aa_train_data_mlops.csv', low_memory=False)

### Cleaning and Choosing Best Data Types

In [2]:
# 1. Converter a coluna `pickup_ts` para datetime
train_df['pickup_ts'] = pd.to_datetime(train_df['pickup_ts'])
train_df['pickup_hour'] = train_df['pickup_ts'].dt.hour  # coletando horário da corrida
train_df['pickup_day_of_the_week'] = train_df['pickup_ts'].dt.day_name()

# 2. Preencher valores vazios de `pickup_airport_code` e `dropoff_airport_code`
train_df['pickup_airport_code'] = train_df['pickup_airport_code'].fillna('Unknown').astype('category')
train_df['dropoff_airport_code'] = train_df['dropoff_airport_code'].fillna('Unknown').astype('category')

# 3. Filtrar linhas inválidas de `driver_rating`
# Remover linhas onde `driver_rating` não pode ser convertido em float (como '\\N')
train_df = train_df[train_df['driver_rating'] != '\\N']

# 4. Transformar `driver_rating` em float
train_df['driver_rating'] = train_df['driver_rating'].astype(float)

#### Weekends
- **Column Added**: `is_weekend`
- **Explanation**: This column is based on `pickup_day_of_week`. Trips on weekends tend to face different traffic conditions compared to weekdays, which may significantly impact trip durations.

#### Outside São Paulo
- **Columns Added**: `pickup_outside_sp`, `dropoff_outside_sp`
- **Explanation**: These columns are based on the geographical boundaries of São Paulo, defined by specific latitude and longitude limits. The function checks whether the pickup or dropoff location is outside São Paulo. This is useful because trips that start or end outside the city may face different road conditions, potentially impacting trip durations.
  - **Latitude Range**: -23.68 (South) to -23.35 (North)
  - **Longitude Range**: -46.83 (West) to -46.40 (East)
- **Why it's useful**: Identifying trips that begin or end outside São Paulo is important for understanding different traffic patterns and road conditions, which can vary significantly from those within the city. These trips might be longer and subject to different regulations or transportation policies, which can affect trip duration predictions.

#### Holidays
- **Column Added**: `is_holiday`
- **Source**: https://portal.inmet.gov.br/dadoshistoricos#:~:text=Instituto%20Nacional%20de%20Meteorologia%20%2D%20INMET
- **Potential Enhancement**: Considering adding a column for **holidays on Fridays or Mondays**. These extended weekends often lead to traffic congestion, and thus, such holidays could have a greater impact on trip duration predictions.

#### Weather
- **Future Consideration**: A `weather_condition` column can capture the impact of adverse weather conditions (e.g., rain or fog) on travel time, as these can cause delays and traffic slowdowns.
- **Source**: https://capital.sp.gov.br/w/noticia/

#### Rush Hour
- **Column Added**: `is_rush_hour`
- **Source**: [Companhia de Engenharia de Tráfego - CET São Paulo](https://www.cetsp.com.br/)
- **Explanation**: This column flags trips that occur during peak traffic times in São Paulo, specifically during **morning rush hour** (6 AM - 9 AM) and **evening rush hour** (5 PM - 8 PM). These periods generally have heavier traffic due to commuters, which can significantly impact trip duration.
- **Why it's useful**: Identifying trips that occur during rush hour is crucial because traffic congestion is typically at its highest during these times. Trips taken during rush hour are more likely to be delayed or experience longer travel times due to increased traffic volume. This flag can help improve model accuracy by considering the impact of traffic congestion during these critical periods.


In [3]:
train_df['is_weekend'] = train_df['pickup_day_of_the_week'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

# Limites geográficos de São Paulo
LAT_MIN = -23.68
LAT_MAX = -23.35
LNG_MIN = -46.83
LNG_MAX = -46.40

# Função para verificar se a coordenada está dentro dos limites de São Paulo
def is_outside_sao_paulo(lat, lng):
    if lat < LAT_MIN or lat > LAT_MAX or lng < LNG_MIN or lng > LNG_MAX:
        return 1  # Fora de São Paulo
    return 0  # Dentro de São Paulo

# Aplicar a função para criar colunas de flags
train_df['pickup_outside_sp'] = train_df.apply(lambda row: is_outside_sao_paulo(row['pick_lat'], row['pick_lng']), axis=1)
train_df['dropoff_outside_sp'] = train_df.apply(lambda row: is_outside_sao_paulo(row['dropoff_lat'], row['dropoff_lng']), axis=1)

holidays_df = pd.read_excel('data_extra/feriados_cidade_de_sao_paulo_2022-2023.xlsx')
holidays_df['date'] = pd.to_datetime(holidays_df['Data'])
train_df['pickup_date'] = pd.to_datetime(train_df['pickup_ts']).dt.date
train_df['is_holiday'] = train_df['pickup_date'].isin(holidays_df['date'].dt.date).astype(int)

def is_rush_hour(hour):
    # Morning rush hour: 6 AM - 9 AM
    # Evening rush hour: 5 PM - 8 PM
    if (6 <= hour <= 9) or (17 <= hour <= 20):
        return 1  # Rush hour
    else:
        return 0  # Non-rush hour

# Apply the function to create a rush hour flag based on the pickup hour
train_df['is_rush_hour'] = train_df['pickup_hour'].apply(is_rush_hour)

In [4]:
def categorize_rain(precipitation):
    if precipitation == 0:
        return 'No rain'
    elif 0 < precipitation <= 2.5:
        return 'Light rain'
    elif 2.5 < precipitation <= 7.6:
        return 'Moderate rain'
    else:
        return 'Heavy rain'

# Load both 2022 and 2023 weather data
weather_2022_df = pd.read_excel('data_extra/dados_meteorologicos_sp_2022.xlsx')
weather_2023_df = pd.read_excel('data_extra/dados_meteorologicos_sp_2023.xlsx')

# Extracting the hour from 'Hora UTC' column (first four characters represent the hour) for both datasets
weather_2022_df['Hora UTC'] = weather_2022_df['Hora UTC'].str[:2].astype(int)
weather_2023_df['Hora UTC'] = weather_2023_df['Hora UTC'].str[:2].astype(int)

# Convert the extracted hour into a timedelta
weather_2022_df['Hora UTC'] = pd.to_timedelta(weather_2022_df['Hora UTC'], unit='h')
weather_2023_df['Hora UTC'] = pd.to_timedelta(weather_2023_df['Hora UTC'], unit='h')

# Combine 'Data' and 'Hora UTC' into a proper UTC datetime column for both datasets
weather_2022_df['datetime_utc'] = weather_2022_df['Data'] + weather_2022_df['Hora UTC']
weather_2023_df['datetime_utc'] = weather_2023_df['Data'] + weather_2023_df['Hora UTC']

# Concatenate 2022 and 2023 weather data into a single dataframe
weather_combined_df = pd.concat([weather_2022_df, weather_2023_df], ignore_index=True)

# Create a 'date_utc' column for merging (use floor to hour precision)
weather_combined_df['date_utc'] = weather_combined_df['datetime_utc'].dt.floor('H')

# Convert 'pickup_ts' in train_df to datetime and adjust to UTC (São Paulo time is UTC-3)
train_df['pickup_ts'] = pd.to_datetime(train_df['pickup_ts'], format='%Y-%m-%d %H:%M:%S')
train_df['pickup_ts_utc'] = train_df['pickup_ts'] - pd.DateOffset(hours=3)

# Create 'date_utc' in train_df by flooring the 'pickup_ts_utc' to the nearest hour
train_df['date_utc'] = train_df['pickup_ts_utc'].dt.floor('H')

# Merge the combined weather data with the train_df
merged_df = pd.merge(train_df, weather_combined_df, on='date_utc', how='left')
train_df = merged_df


In [5]:
# Categorize rain intensity
train_df['rain_category'] = train_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].apply(categorize_rain)

# Total rain accumulation over the last 1 and 6 hours
train_df['rain_last_hour'] = train_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].rolling(window=2, min_periods=1).sum()
train_df['rain_last_6_hours'] = train_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].rolling(window=6, min_periods=1).sum()

# Rain intensity change
train_df['rain_intensity_change'] = train_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].diff()

# Combining rain and wind speed
train_df['rain_wind_combined'] = train_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] * train_df['VENTO, VELOCIDADE HORARIA (m/s)']

# Creating lagged rain features
train_df['rain_lag_1'] = train_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].shift(1)
train_df['rain_lag_2'] = train_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].shift(2)

# Optimizing weighted rain feature computation using vectorized operations instead of .apply()
weights = [0.5, 0.3, 0.2]  # Giving more importance to recent hours
train_df['weighted_rain'] = train_df['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'].rolling(window=3, min_periods=1).apply(
    lambda x: np.dot(weights[-len(x):], x)  # Apply dot product with truncated weights
)


In [6]:
falhas_df = pd.read_excel('data_extra/failures_public_transport.xlsx')

# Convert 'date' to datetime for merging
falhas_df['date'] = pd.to_datetime(falhas_df['date'])

# Convert 'pickup_ts' to datetime, extract the date part, and ensure it is in datetime64[ns]
train_df['date'] = pd.to_datetime(train_df['pickup_ts']).dt.floor('D')

# Ensure 'train_df["date"]' is in datetime64[ns] format
train_df['date'] = pd.to_datetime(train_df['date'])

# Merge the trip data with the failures data on the 'date'
merged_with_failures = pd.merge(train_df, falhas_df, left_on='date', right_on='date', how='left')

# Flag failures only if the trip occurred between 4 AM and midnight
def flag_failure(hour, event):
    if pd.isnull(event):
        return 0  # No failure
    elif 4 <= hour < 24:  # Operational hours: 4 AM to midnight
        return 1  # Failure during operational hours
    return 0

# Apply the function to create the failure flag
merged_with_failures['failure_flag'] = merged_with_failures.apply(
    lambda row: flag_failure(row['pickup_ts'].hour, row['event']), axis=1
)

In [7]:
# remove some columns
train_df = train_df[[
    'uuid',
    'pickup_ts', 'pickup_day_of_the_week', 'pickup_hour', 'pick_lat', 'pick_lng', 'pickup_outside_sp',
    'is_weekend', 'is_holiday', 'is_rush_hour',
    'dropoff_ts', 'dropoff_lat', 'dropoff_lng', 'dropoff_outside_sp',
    'eta', 'ata',
    'trip_distance', 'trip_duration',
    'is_airport', 'pickup_airport_code', 'dropoff_airport_code',
    'is_surged', 'surge_multiplier',
    'driver_rating', 'lifetime_trips',
    'rain_category', 'rain_last_hour', 'rain_last_6_hours', 'rain_intensity_change', 'rain_wind_combined', 'rain_lag_1', 'rain_lag_2', 'weighted_rain'
    ]]
print(train_df.shape)
train_df.head()
train_df.to_csv('data_fnal/train_df_cleaned.csv')

(1499996, 33)


,uuid,pickup_ts,pickup_day_of_the_week,pickup_hour,pick_lat,pick_lng,pickup_outside_sp,is_weekend,is_holiday,is_rush_hour,...,driver_rating,lifetime_trips,rain_category,rain_last_hour,rain_last_6_hours,rain_intensity_change,rain_wind_combined,rain_lag_1,rain_lag_2,weighted_rain
0,2d1cd1ea-1874-4e27-9bd5-aa599cb083d9,2023-03-09 07:50:28,Thursday,7,-23.518545,-46.207977,1,0,0,1,...,4.99,19095,No rain,0.0,0.0,NaN,0.00,NaN,NaN,0.00
1,2b7a0c2f-59d1-43a1-9c31-a13e2bbdfb82,2023-01-31 20:26:33,Tuesday,20,-23.428263,-46.533930,0,0,0,1,...,4.99,1811,No rain,0.0,0.0,0.0,0.00,0.0,NaN,0.00
2,9371a38d-4135-46c3-aa0f-bbf1a52f32a8,2023-02-10 13:14:42,Friday,13,-23.622234,-46.661434,0,0,0,0,...,4.99,1795,No rain,0.0,0.0,0.0,0.00,0.0,0.0,0.00
3,f1739782-6ccb-4d80-a88d-fdd4af7807e9,2022-12-04 17:19:41,Sunday,17,-23.665737,-46.649395,0,1,0,1,...,4.95,14415,Light rain,0.2,0.2,0.2,0.62,0.0,0.0,0.04
4,9f8ad376-877f-4c75-8263-72a9aa02c0fa,2023-02-16 07:24:01,Thursday,7,-23.505342,-46.292507,1,0,0,1,...,4.94,2168,No rain,0.2,0.2,-0.2,0.00,0.2,0.0,0.06
